In [47]:
import pandas as pd
customer = pd.read_csv('customer_join.csv')
uselog_months = pd.read_csv('use_log_months.csv')

# 데이터 전처리
전 장처럼 과거 6개월간의 데이터로 미래를 예측하면 최근 5개월 이내에 가입한 회원에 대한 예측은 할 수 없기 때문에 이번 달과 전 달을 이용

In [48]:
year_months = list(uselog_months['연월'].unique())
uselog= pd.DataFrame()
for i in range(1, len(year_months)):
    tmp = uselog_months.loc[uselog_months['연월']==year_months[i]]
    tmp.rename(columns={"count":"count_0"}, inplace=True)
    tmp_before = uselog_months.loc[uselog_months['연월']==year_months[i-1]]
    del tmp_before['연월']
    tmp_before.rename(columns={"count":"count_1"},inplace=True)
    tmp = pd.merge(tmp, tmp_before,on='customer_id', how='left')
    uselog = pd.concat([uselog, tmp], ignore_index=True)
uselog.head()

C:\Users\user\anaconda3\lib\site-packages\pandas\core\frame.py:4125: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  return super().rename(


,연월,customer_id,count_0,count_1
0,201805,AS002855,5,4.0
1,201805,AS009373,4,3.0
2,201805,AS015233,7,NaN
3,201805,AS015315,3,6.0
4,201805,AS015739,5,7.0


# 탈퇴 전월의 탈퇴 고객 데이터를 작성하자
이 스포츠 센터의 탈퇴 절차는 탈퇴신청을 하면 다음 달 말에 탈퇴가 되는 것이기 때문에 end_date를 사용하면 안되고 end_date의 전월과 탈퇴 신청을 한 월의 1개월 전의 데이터를 이용해서 예측을 해야 탈퇴 신청 예측을 할 수 있다.


In [49]:
from dateutil.relativedelta import relativedelta
exit_customer = customer.loc[customer['is_deleted']==1]
exit_customer['exit_date']=None
exit_customer['end_date']=pd.to_datetime(exit_customer['end_date'])
for i in range(len(exit_customer)):
    exit_customer['exit_date'].iloc[i] = exit_customer['end_date'].iloc[i]-relativedelta(months=1)
exit_customer['연월'] = exit_customer['exit_date'].dt.strftime("%Y%m")
uselog['연월'] = uselog['연월'].astype(str)
exit_uselog = pd.merge(uselog, exit_customer, on=['customer_id','연월'], how='left')
exit_uselog

<ipython-input-49-c422a1a7b413>:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  exit_customer['exit_date']=None
<ipython-input-49-c422a1a7b413>:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  exit_customer['end_date']=pd.to_datetime(exit_customer['end_date'])
C:\Users\user\anaconda3\lib\site-packages\pandas\core\indexing.py:671: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user

,연월,customer_id,count_0,count_1,name,class,gender,start_date,end_date,campaign_id,...,campaign_name,mean,median,max,min,routine_flag,calc_date,mambership_period,membership_period,exit_date
0,201805,AS002855,5,4.0,NaN,NaN,NaN,NaN,NaT,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,201805,AS009373,4,3.0,NaN,NaN,NaN,NaN,NaT,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,201805,AS015233,7,NaN,NaN,NaN,NaN,NaN,NaT,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,201805,AS015315,3,6.0,NaN,NaN,NaN,NaN,NaT,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,201805,AS015739,5,7.0,NaN,NaN,NaN,NaN,NaT,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
33846,201903,TS995853,8,11.0,NaN,NaN,NaN,NaN,NaT,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
33847,201903,TS998593,8,7.0,NaN,NaN,NaN,NaN,NaT,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
33848,201903,TS999079,3,2.0,NaN,NaN,NaN,NaN,NaT,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
33849,201903,TS999231,6,6.0,NaN,NaN,NaN,NaN,NaT,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [50]:
exit_uselog = exit_uselog.dropna(subset=['name'])
print(len(exit_uselog))
print(len(exit_uselog["customer_id"].unique()))
exit_uselog.head()

1104
1104


,연월,customer_id,count_0,count_1,name,class,gender,start_date,end_date,campaign_id,...,campaign_name,mean,median,max,min,routine_flag,calc_date,mambership_period,membership_period,exit_date
19,201805,AS055680,3,3.0,XXXXX,C01,M,2018-03-01,2018-06-30,CA1,...,2_일반,3.000000,3.0,3.0,3.0,0.0,2018-06-30,0.0,3.0,2018-05-30 00:00:00
57,201805,AS169823,2,3.0,XX,C01,M,2017-11-01,2018-06-30,CA1,...,2_일반,3.000000,3.0,4.0,2.0,1.0,2018-06-30,0.0,7.0,2018-05-30 00:00:00
110,201805,AS305860,5,3.0,XXXX,C01,M,2017-06-01,2018-06-30,CA1,...,2_일반,3.333333,3.0,5.0,2.0,0.0,2018-06-30,0.0,12.0,2018-05-30 00:00:00
128,201805,AS363699,5,3.0,XXXXX,C01,M,2018-02-01,2018-06-30,CA1,...,2_일반,3.333333,3.0,5.0,2.0,0.0,2018-06-30,0.0,4.0,2018-05-30 00:00:00
147,201805,AS417696,1,4.0,XX,C03,F,2017-09-01,2018-06-30,CA1,...,2_일반,2.000000,1.0,4.0,1.0,0.0,2018-06-30,0.0,9.0,2018-05-30 00:00:00


### 지속 회원의 데이터를 작성하자
탈퇴 데이터가 1104개밖에 없기 때문에 지속회원 데이터 27422개를 전부 사용한다면 불균형 데이터가 될 것이다. 따라서 지속 회원 데이터도 회원당 1개가 되게 언더샘플링한다. 즉, 2018년 5월 A씨와 2018년 12월 A씨 중 하나만 선택한다.


In [51]:
conti_customer = customer.loc[customer['is_deleted']==0]
conti_uselog = pd.merge(uselog, conti_customer, on=['customer_id'],how='left')
print(len(conti_uselog))
conti_uselog = conti_uselog.dropna(subset=["name"])
print(len(conti_uselog))

33851
27422


In [52]:
conti_uselog = conti_uselog.sample(frac=1).reset_index(drop=True)
conti_uselog = conti_uselog.drop_duplicates(subset="customer_id")
print(len(conti_uselog))
conti_uselog.head()

2842


,연월,customer_id,count_0,count_1,name,class,gender,start_date,end_date,campaign_id,...,price,campaign_name,mean,median,max,min,routine_flag,calc_date,mambership_period,membership_period
0,201805,GD172092,8,9.0,XXX,C02,F,2017-11-01,NaN,CA1,...,7500.0,2_일반,6.250000,6.5,9.0,1.0,1.0,2019-04-30,0.0,17.0
1,201805,HI106422,6,7.0,XXX,C01,M,2016-12-01,NaN,CA1,...,10500.0,2_일반,5.166667,4.5,10.0,2.0,1.0,2019-04-30,0.0,28.0
2,201811,GD437531,7,2.0,XXXXX,C01,M,2015-06-01,NaN,CA1,...,10500.0,2_일반,4.416667,4.5,7.0,1.0,1.0,2019-04-30,0.0,46.0
3,201807,AS335390,4,7.0,XX,C01,F,2017-01-01,NaN,CA1,...,10500.0,2_일반,5.750000,6.0,7.0,4.0,1.0,2019-04-30,0.0,27.0
4,201807,GD126264,5,7.0,XXXXX,C03,M,2017-02-01,NaN,CA1,...,6000.0,2_일반,5.083333,5.0,7.0,2.0,1.0,2019-04-30,0.0,26.0


In [53]:
predict_data = pd.concat([conti_uselog, exit_uselog],ignore_index=True)
print(len(predict_data))
predict_data.head()

3946


,연월,customer_id,count_0,count_1,name,class,gender,start_date,end_date,campaign_id,...,campaign_name,mean,median,max,min,routine_flag,calc_date,mambership_period,membership_period,exit_date
0,201805,GD172092,8,9.0,XXX,C02,F,2017-11-01,NaN,CA1,...,2_일반,6.250000,6.5,9.0,1.0,1.0,2019-04-30,0.0,17.0,NaN
1,201805,HI106422,6,7.0,XXX,C01,M,2016-12-01,NaN,CA1,...,2_일반,5.166667,4.5,10.0,2.0,1.0,2019-04-30,0.0,28.0,NaN
2,201811,GD437531,7,2.0,XXXXX,C01,M,2015-06-01,NaN,CA1,...,2_일반,4.416667,4.5,7.0,1.0,1.0,2019-04-30,0.0,46.0,NaN
3,201807,AS335390,4,7.0,XX,C01,F,2017-01-01,NaN,CA1,...,2_일반,5.750000,6.0,7.0,4.0,1.0,2019-04-30,0.0,27.0,NaN
4,201807,GD126264,5,7.0,XXXXX,C03,M,2017-02-01,NaN,CA1,...,2_일반,5.083333,5.0,7.0,2.0,1.0,2019-04-30,0.0,26.0,NaN


### 예측할 달의 재적 기간을 작성하자

In [54]:
predict_data['period']=0
predict_data['now_date']=pd.to_datetime(predict_data['연월'],format="%Y%m")
predict_data['start_date']=pd.to_datetime(predict_data['start_date'])
for i in range(len(predict_data)):
    delta = relativedelta(predict_data['now_date'][i], predict_data['start_date'][i])
    predict_data['period'][i] = int(delta.years*12+delta.months)
predict_data.head()

<ipython-input-54-5b3b38f94739>:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  predict_data['period'][i] = int(delta.years*12+delta.months)


,연월,customer_id,count_0,count_1,name,class,gender,start_date,end_date,campaign_id,...,median,max,min,routine_flag,calc_date,mambership_period,membership_period,exit_date,period,now_date
0,201805,GD172092,8,9.0,XXX,C02,F,2017-11-01,NaN,CA1,...,6.5,9.0,1.0,1.0,2019-04-30,0.0,17.0,NaN,6,2018-05-01
1,201805,HI106422,6,7.0,XXX,C01,M,2016-12-01,NaN,CA1,...,4.5,10.0,2.0,1.0,2019-04-30,0.0,28.0,NaN,17,2018-05-01
2,201811,GD437531,7,2.0,XXXXX,C01,M,2015-06-01,NaN,CA1,...,4.5,7.0,1.0,1.0,2019-04-30,0.0,46.0,NaN,41,2018-11-01
3,201807,AS335390,4,7.0,XX,C01,F,2017-01-01,NaN,CA1,...,6.0,7.0,4.0,1.0,2019-04-30,0.0,27.0,NaN,18,2018-07-01
4,201807,GD126264,5,7.0,XXXXX,C03,M,2017-02-01,NaN,CA1,...,5.0,7.0,2.0,1.0,2019-04-30,0.0,26.0,NaN,17,2018-07-01


### 결측치 제거

In [55]:
predict_data.isna().sum()

연월                      0
customer_id             0
count_0                 0
count_1               248
name                    0
class                   0
gender                  0
start_date              0
end_date             2842
campaign_id             0
is_deleted              0
class_name              0
price                   0
campaign_name           0
mean                    0
median                  0
max                     0
min                     0
routine_flag            0
calc_date               0
mambership_period       0
membership_period       0
exit_date            2842
period                  0
now_date                0
dtype: int64

In [56]:
predict_data = predict_data.dropna(subset=['count_1'])
predict_data.isna().sum()

연월                      0
customer_id             0
count_0                 0
count_1                 0
name                    0
class                   0
gender                  0
start_date              0
end_date             2646
campaign_id             0
is_deleted              0
class_name              0
price                   0
campaign_name           0
mean                    0
median                  0
max                     0
min                     0
routine_flag            0
calc_date               0
mambership_period       0
membership_period       0
exit_date            2646
period                  0
now_date                0
dtype: int64

### 문자열 변수를 처리할 수 있게 가공하자
캠페인 구분, 회원구분, 성별과 같은 문자열 데이터를 가테고리 변수라고 한다. 이런 데이터도 머신러닝을 하는 데에 있어 중요한 변수가 된다.
설명 변수로는 1개월 전의 이용횟수 count_1, 카테고리 변수인 campaign, name,class_name, gender, routine_flag, period를 사용. 목적 변수로 탈퇴 플래그인 is_deleted를 사용

In [57]:
target_col = ['campaign_name','class_name','gender','count_1','routine_flag','period','is_deleted']
predict_data = predict_data[target_col]
print(len(predict_data))
predict_data.head()

3698


,campaign_name,class_name,gender,count_1,routine_flag,period,is_deleted
0,2_일반,1_주간,F,9.0,1.0,6,0.0
1,2_일반,0_종일,M,7.0,1.0,17,0.0
2,2_일반,0_종일,M,2.0,1.0,41,0.0
3,2_일반,0_종일,F,7.0,1.0,18,0.0
4,2_일반,2_야간,M,7.0,1.0,17,0.0


In [58]:
#더미 변수
predict_data = pd.get_dummies(predict_data)
predict_data.head()

,count_1,routine_flag,period,is_deleted,campaign_name_0_입회비반액할인,campaign_name_1_입회비무료,campaign_name_2_일반,class_name_0_종일,class_name_1_주간,class_name_2_야간,gender_F,gender_M
0,9.0,1.0,6,0.0,0,0,1,0,1,0,1,0
1,7.0,1.0,17,0.0,0,0,1,1,0,0,0,1
2,2.0,1.0,41,0.0,0,0,1,1,0,0,0,1
3,7.0,1.0,18,0.0,0,0,1,1,0,0,1,0
4,7.0,1.0,17,0.0,0,0,1,0,0,1,0,1


In [59]:
#다중공선성 해결을 위해 컬럼 하나씩을 지워야 한다.
del predict_data['campaign_name_2_일반']
del predict_data['class_name_2_야간']
del predict_data['gender_M']
predict_data.head()

,count_1,routine_flag,period,is_deleted,campaign_name_0_입회비반액할인,campaign_name_1_입회비무료,class_name_0_종일,class_name_1_주간,gender_F
0,9.0,1.0,6,0.0,0,0,0,1,1
1,7.0,1.0,17,0.0,0,0,1,0,0
2,2.0,1.0,41,0.0,0,0,1,0,0
3,7.0,1.0,18,0.0,0,0,1,0,1
4,7.0,1.0,17,0.0,0,0,0,0,0


# 의사결정 트리를 사용해서 탈퇴 예측 모델을 구축

In [60]:
from sklearn.tree import DecisionTreeClassifier
import sklearn.model_selection

exit = predict_data.loc[predict_data['is_deleted']==1]
conti = predict_data.loc[predict_data['is_deleted']==0].sample(len(exit))

X = pd.concat([exit,conti],ignore_index=True)
y = X['is_deleted']
del X['is_deleted']
X_train, X_test, y_train, y_test = sklearn.model_selection.train_test_split(X,y)

model = DecisionTreeClassifier(random_state=0)
model.fit(X_train, y_train)
y_test_pred = model.predict(X_test)
print(y_test_pred)

[1. 1. 0. 0. 1. 1. 0. 1. 1. 1. 0. 1. 1. 1. 1. 1. 0. 1. 0. 1. 1. 0. 0. 1.
 1. 0. 1. 1. 0. 1. 0. 0. 1. 1. 1. 0. 1. 0. 0. 1. 0. 0. 0. 0. 1. 0. 1. 1.
 0. 0. 0. 1. 1. 1. 0. 1. 0. 0. 1. 1. 0. 1. 0. 1. 1. 0. 1. 0. 1. 0. 0. 0.
 1. 0. 1. 1. 0. 1. 0. 0. 0. 0. 1. 0. 0. 0. 0. 0. 0. 1. 1. 0. 1. 1. 0. 1.
 0. 0. 1. 0. 1. 0. 0. 1. 1. 1. 0. 0. 1. 1. 0. 0. 1. 0. 0. 0. 0. 0. 0. 0.
 1. 0. 0. 0. 0. 0. 0. 1. 1. 1. 0. 1. 1. 1. 0. 1. 1. 0. 0. 0. 0. 0. 1. 1.
 0. 0. 0. 1. 1. 0. 0. 1. 0. 0. 0. 0. 0. 1. 1. 0. 1. 1. 1. 1. 1. 1. 0. 0.
 1. 1. 1. 0. 1. 0. 0. 1. 1. 1. 0. 1. 0. 1. 0. 1. 0. 0. 0. 1. 0. 0. 1. 1.
 1. 1. 1. 0. 0. 1. 0. 1. 1. 0. 1. 0. 0. 1. 0. 0. 0. 0. 0. 1. 0. 1. 1. 0.
 0. 1. 0. 0. 0. 0. 1. 0. 0. 1. 0. 0. 1. 1. 1. 0. 0. 1. 0. 0. 0. 1. 0. 1.
 1. 1. 0. 1. 1. 0. 1. 0. 1. 0. 0. 0. 1. 1. 1. 0. 1. 1. 0. 1. 0. 1. 0. 1.
 1. 1. 1. 0. 0. 1. 1. 0. 1. 1. 1. 1. 1. 1. 1. 1. 1. 0. 1. 1. 1. 1. 1. 1.
 1. 0. 1. 0. 1. 1. 1. 0. 0. 1. 1. 0. 0. 0. 0. 1. 1. 0. 1. 0. 0. 0. 0. 0.
 1. 0. 1. 1. 0. 0. 1. 0. 1. 0. 1. 0. 1. 1. 1. 0. 1.

In [61]:
results_test = pd.DataFrame({'y_test':y_test, 'y_pred':y_test_pred})
results_test.head()

,y_test,y_pred
681,1.0,1.0
435,1.0,1.0
1845,0.0,0.0
1204,0.0,0.0
132,1.0,1.0


### 예측 모델을 평가하고 모델을 튜닝

In [62]:
correct = len(results_test.loc[results_test['y_test']==results_test['y_pred']])
data_count = len(results_test)
score_test= correct/data_count
print(score_test)

0.9068441064638784


In [63]:
print(model.score(X_test,y_test))
print(model.score(X_train, y_train))

0.9068441064638784
0.9778200253485425


### 학습용 데이터에 너무 맞춘 과적합 경향을 해결하기 위해 의사결정나무의 깊이를 5로 제한하여 단순한 모델을 만든다.

In [64]:
X = pd.concat([exit,conti],ignore_index=True)
y = X['is_deleted']
del X['is_deleted']
X_train, X_test, y_train, y_test = sklearn.model_selection.train_test_split(X,y)

model = DecisionTreeClassifier(random_state=0, max_depth=5)
model.fit(X_train, y_train)
print(model.score(X_test,y_test))
print(model.score(X_train, y_train))

0.8973384030418251
0.9309252217997465


In [65]:
importance = pd.DataFrame({'feature_names':X.columns, 'coefficient':model.feature_importances_})
importance

,feature_names,coefficient
0,count_1,0.325290
1,routine_flag,0.125955
2,period,0.541006
3,campaign_name_0_입회비반액할인,0.000210
4,campaign_name_1_입회비무료,0.000000
5,class_name_0_종일,0.000000
6,class_name_1_주간,0.007289
7,gender_F,0.000250


# 회원 탈퇴를 예측하자

In [66]:
count_1 = 3
routine_flag = 1
period= 10
campaign_name = "입회비무료"
class_name = "종일"
gender='M'

In [67]:
if campaign_name == "입회비반값할인":
    campaign_name_list = [1,0]
elif campaign_name == "입회비무료":
    campaign_name_list = [0,1]
elif campaign_name=='일반':
    campaign_name_list=[0,0]
if class_name =='종일':
    class_name_list = [1,0]
elif class_name=="주간":
    class_name_list = [0,1]
elif class_name=='야간':
    class_name_list=[0,0]
if gender=='F':
    gender_list=[1]
elif gender=='M':
    gender_list=[0]
input_data = [count_1, routine_flag, period]
input_data.extend(campaign_name_list)
input_data.extend(class_name_list)
input_data.extend(gender_list)

In [68]:
print(model.predict([input_data]))
print(model.predict_proba([input_data]))

[1.]
[[0.00222222 0.99777778]]
